In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import utils
import globvars as gv
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


rawname_ = 'S01_off_hold'
rawname_ = 'S01_off_move'
rawname_ = 'S01_on_hold'
rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'
rawname_ = 'S02_off_move'  
#rawname_ = 'S02_on_hold' 
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_move'  
#rawname_ = 'S03_off_hold'  

#rawname = rawname_ + '_resample_raw.fif'
rawname = rawname_ + '_resample_afterICA_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    

from mne.decoding import SPoC
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_predict

from mne import Epochs

In [ ]:
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

rectconv_fname_full = os.path.join(gv.data_dir, '{}_emg_rectconv.fif'.format(rawname_) )
rectconvraw  = mne.io.read_raw_fif(rectconv_fname_full)

anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
anns = mne.read_annotations(anns_fn_full)

rectconvraw.set_annotations(anns)

anns_MEGartif_fn = rawname_ + '_ann_MEGartif.txt'
anns_MEGartif_fn_full = os.path.join(data_dir, anns_MEGartif_fn)
anns_MEGartif = mne.read_annotations(anns_MEGartif_fn_full)

rectconvraw.annotations.append(anns_MEGartif.onset, anns_MEGartif.duration, anns_MEGartif.description)

subj,medcond,task  = utils.getParamsFromRawname(rawname_)
sind_str = subj
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)
maintremside = gen_subj_info[subj]['tremor_side']

In [ ]:
sometrem = 0
for ann_descr in anns.description:
    if ann_descr.startswith('trem_{}'.format(maintremside[0].upper())):
        sometrem = 1
assert sometrem

In [ ]:

print(maintremside)
#emg_ps = utils.getEMGperHand(rectconvraw)
emg_rot_ps = utils.getMainEMGcomp(rectconvraw)
emg = emg_rot_ps[maintremside]

megraw = raw.copy()
megraw.load_data()
megraw.pick_types(meg=True)

In [ ]:
task_start = utils.findTaskStart(raw.annotations)
task_offset = 5 # in sec
print( task_start - task_offset )
megraw.crop(tmin=0, tmax = task_start - task_offset)
emg.crop(tmin = 0, tmax = task_start - task_offset)

print( rawname_ )

%matplotlib qt
emg.plot(scalings={'eeg':'auto'});



In [ ]:
megraw.set_annotations(emg.annotations)

#emg.set_annotations(megraw.annotations)

print( megraw.info['bads'], emg.info['bads'] )
print(megraw.annotations, emg.annotations)

assert 0 == len(emg.times) - len(megraw.times)

print( megraw.info['bads'] )

In [ ]:
help(SPoC)
help(cross_val_predict)

In [ ]:
import scipy

In [ ]:
# fband_to_use = 'tremor'
# fband_to_use = 'beta'
# fband_to_use = 'gamma'

N = 2
N_crossval = 2
for fband_to_use in ['tremor', 'beta', 'gamma']:
#for fband_to_use in ['gamma']:
    fb = gv.fbands[fband_to_use]
    megraw_flt = megraw.copy().filter(l_freq=fb[0], h_freq=fb[1])

    evt_dur_s = 1. # orig 0.25
    epoch_dur_s = 2   # orig 1.5
    decim = 1

    # Build epochs as sliding windows over the continuous raw file
    events = mne.make_fixed_length_events(megraw, id=1, duration=evt_dur_s)

    # Epoch length is 1.5 second
    meg_epochs = Epochs(megraw_flt, events, tmin=0., tmax=epoch_dur_s, baseline=None,
                        detrend=1, decim=decim)
    emg_epochs = Epochs(emg, events, tmin=0., tmax=epoch_dur_s, baseline=None)

    # Prepare classification
    #rr = meg_epochs._get_data(verbose=None)

    X = meg_epochs.get_data()
    y = emg_epochs.get_data().var(axis=2)[:, 0]  # target is EMG power
    import gc; gc.collect()


    # Classification pipeline with SPoC spatial filtering and Ridge Regression
    spoc = SPoC(n_components=N, log=True, reg='oas', rank='full')
    clf = make_pipeline(spoc, Ridge())
    # Define a two fold cross-validation
    cv = KFold(n_splits=N_crossval, shuffle=False)

    try:
        # Run cross validaton
        y_preds = cross_val_predict(clf, X, y, cv=cv, n_jobs = 10)
    except scipy.linalg.LinAlgError as e:
        print(str(e))
        continue

    # Plot the True EMG power and the EMG power predicted from MEG data

    fig, ax = plt.subplots(1, 1, figsize=[10, 4])
    times = megraw.times[meg_epochs.events[:, 0] - megraw.first_samp]

    ax.plot(times, y_preds, color='b', label='Predicted EMG')
    ax.plot(times, y, color='r', label='True EMG')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('EMG Power')
    ax.set_title('SPoC MEG Predictions band={}'.format(fband_to_use))
    plt.legend()
    mne.viz.tight_layout()
    plt.show()

    figname = '{}_SPoC_{}.pdf'.format(rawname_,fband_to_use)
    plt.savefig(os.path.join(gv.dir_fig,figname) )
    plt.close()

    spoc.fit(X, y)
    spoc.plot_patterns(meg_epochs.info);

    figname = '{}_SPoC_patterns_{}.pdf'.format(rawname_,fband_to_use)
    plt.savefig(os.path.join(gv.dir_fig,figname) )
    plt.close()